In [7]:
#Import libraries

%matplotlib qt

import os, sys
    

import pandas as pd
import numpy as np
import scipy.io

import matplotlib
import matplotlib.cm as cm
import matplotlib.pyplot as plt

matplotlib.rcParams.update({'figure.max_open_warning': 0})

import seaborn as sns


import networkx as nx


import basefunction.anass_load as bf
import basefunction.bk_load as bk

In [8]:
def draw_node(df,idx_type):
    
    #Author : BK 7/2020
    # Input df and idx as returned by get_hubs()
    # Output is a plot of the hubs
    
    
    G = nx.DiGraph()
    
    cm = plt.get_cmap('gist_rainbow')
    for i,node in enumerate(np.unique([df['pre_syn'].values,df['post_syn'].values])):
        if idx_type[i,3] == 1:
            G.add_node(node,color = cm(i/len(df)),id = node,Type = 'Pyr')
        elif idx_type[i,3] == 2:
            G.add_node(node,color = cm(i/len(df)),id = node,Type = 'Int')
        else:
            G.add_node(node,color = cm(i/len(df)),id = node,Type = 'Unknown')
#     for i,node in enumerate(np.unique([df['pre_syn'].values,df['post_syn'].values])):
#         G.add_node(node,color = colors[i])
    
    
    
    pyr_nodes = [n for (n,ty) in nx.get_node_attributes(G,'Type').items() if ty == 'Pyr']
    int_nodes = [n for (n,ty) in nx.get_node_attributes(G,'Type').items() if ty == 'Int']
    other_nodes = list(set(G.nodes()) - set(pyr_nodes) - set(int_nodes))
    
#     print(G.nodes())
#     print(pyr_nodes)
#     print(int_nodes)
#     print(other_nodes)

#  Print to check data
    for edge in zip(df['pre_syn'],df['post_syn']):
        syn_type = df['value'][(df['pre_syn'] == edge[0]) & (df['post_syn'] == edge[1])].values[0]
        if syn_type == 'exc':
            color = 'Red'
        elif syn_type == 'inh':
            color = 'Blue'
        G.add_edge(*edge,color = color)   

    node_colors = nx.get_node_attributes(G,'color')
    edge_colors = nx.get_edge_attributes(G, 'color')
    node_shapes = nx.get_node_attributes(G,'shape')


    pos = nx.spring_layout(G,k = 1)
    
    
    nx.draw_networkx_nodes(G,pos,
                           nodelist=pyr_nodes,
                           node_size = 500,
                           node_color = list([node_colors[i] for i in pyr_nodes]),
                           node_shape = '^')
    nx.draw_networkx_nodes(G,pos,
                           nodelist=int_nodes,
                           node_color = list([node_colors[i] for i in int_nodes]),
                           node_size = 500,
                           node_shape = 'o')
    nx.draw_networkx_nodes(G,pos,
                           nodelist=other_nodes,
                           node_color = list([node_colors[i] for i in int_nodes]),
                           node_size = 500,
                           node_shape = 'd')
    nx.draw(G,
            pos,
            with_labels = True,
            nodelist = [],
            connectionstyle='arc3, rad = 0.3', 
            edge_list = G.edges(),
            edge_color = edge_colors.values(),
            alpha = 0.8,
            width=2.0, 
            edge_cmap=plt.cm.Set2)
    
    
def get_hubs():
    #Author : BK 12/08/20
    #Input : path to session
    #Output :
    # df : DataFrame of the hubs
    # idx : np.array of correspondance between shank id vs id in the whole session
    # hubs : np.array of cells in center of hubs
    
    
#     session_index = pd.read_csv('Z:/All-Rats/Billel/session_indexing.csv',sep = ';')

#     session = path.split('\\')[2]
#     os.chdir(path)
    
    
    #Need to run bk.set_current_session first.
    try: bk.session 
    except NameError: bk.session = None
    
    
    #Load matlab variables
    MonoSynConvClick = scipy.io.loadmat(bk.session +'-MonoSynConvClick.mat') 
    activity = scipy.io.loadmat('Billel/Transitions/NREM_REM_ALL.mat')['binned']['activity'][0][0]
    metadata = scipy.io.loadmat('Billel/Transitions/NREM_REM_ALL.mat')['binned']['metadata'][0][0]
    finalType = scipy.io.loadmat('Z:/All-Rats/AllRats-FinalType.mat')['finalType']

    BLA_Shank = bf.get_structure_shanks(bk.session,'BLA')[0] #Get Shank for amygdala

    #Clean finalType in order to have only good day and good rat
    finalType = finalType[(finalType[:,0] == bk.rat) & (finalType[:,1] == bk.day),2:5]
    
    #Get Idx variable in MonoSynConvClick /!\ There is some weird things going on here : 
    idx = MonoSynConvClick['Idx']
    idx = np.hstack((idx[:,:],np.array([finalType[np.all((i[0:2] == finalType[:,0:2]),1),2] for i in idx])))
    # Add type columns to idx from finalType
    
    
    #Keep only BLA Cells 
    inh_pre_syn_shank = MonoSynConvClick['FinalInhMonoSyn'][:,0]
    inh_str_mask = [x in BLA_Shank for x in inh_pre_syn_shank]
    
    inh_pre_syn = MonoSynConvClick['FinalInhMonoSynID'][inh_str_mask,0]
    inh_post_syn = MonoSynConvClick['FinalInhMonoSynID'][inh_str_mask,1]
    type_inh_syn = ['inh']*len(inh_pre_syn)

    exc_pre_syn_shank = MonoSynConvClick['FinalExcMonoSyn'][:,0]
    exc_str_mask = [x in BLA_Shank for x in exc_pre_syn_shank]
    exc_pre_syn = MonoSynConvClick['FinalExcMonoSynID'][exc_str_mask,0]
    exc_post_syn = MonoSynConvClick['FinalExcMonoSynID'][exc_str_mask,1]
    type_exc_syn = ['exc']*len(exc_pre_syn)

    
    type_syn = np.append(type_inh_syn,type_exc_syn)
    pre_syn = np.append(inh_pre_syn,exc_pre_syn)
    post_syn = np.append(inh_post_syn,exc_post_syn)



    # Construct a DataFrame because it is way more elegant
    df = pd.DataFrame({'pre_syn':pre_syn,'post_syn':post_syn,'value':type_syn})
    df['value']=pd.Categorical(df['value'])
    
    #Keep only inh cells with more than 2 mono-syn connexion.
    
    hubs = []
    for i in np.unique([df['pre_syn'].values,df['post_syn'].values]):
        if (len(df[df['pre_syn'] == i])>2) & any(df['value'][df['pre_syn'] == i] == 'inh'):
            hubs.append(i)
    
    print('Found ' + str(len(hubs)) + ' hubs')
    
    return df,idx,hubs

def get_activity():
    
    # Check for set_current_session
    
    try: bk.session 
    except NameError: bk.session = None
        
    activity = scipy.io.loadmat('Billel/Transitions/NREM_REM_ALL.mat')['binned']['activity'][0][0]
    metadata = scipy.io.loadmat('Billel/Transitions/NREM_REM_ALL.mat')['binned']['metadata'][0][0]
    
    return activity,metadata

def draw_hubs(df,idx,hubs,activity,metadata,plot = True):
    # Author : BK 12/08/20
    # Input : df,idx,hubs as given by get_hubs(session) and activity,metadata given by get_activity
    # Output : it's a plot
    
    cm = plt.get_cmap('gist_rainbow')
    
    for hub in hubs:
        df_hub = df[(df['pre_syn'] == hub) | (df['post_syn'] == hub)]
#         df_hub = df
        repeat = df_hub.pivot_table(index=['pre_syn'],aggfunc='size')
        center = np.argmax(repeat)
        center = repeat.index[center]
#         print(str(center)+' Should be the center of graph')

        l = np.unique([df_hub['pre_syn'].values,df_hub['post_syn'].values])
        idx_type = idx[[x in l for x in idx[:,2]]]

        
        if plot:
            plt.figure()
            plt.suptitle(bk.session)
            plt.subplot(1,2,1)
            for c,i in enumerate(np.unique([df_hub['pre_syn'].values,df_hub['post_syn'].values])):
                n_mask = metadata[:,4] == i
                norm = np.mean(activity[20:30,n_mask,:])
                plt.plot((np.mean(activity[:,n_mask,:],2)/1),color = cm(c/len(df_hub)))
        #         plt.plot((v[:,17]),color = cm(c/len(df_hub)))

            plt.axvspan(30,41, facecolor='darkorange', alpha=0.5)
            plt.title('Activity')
            plt.ylabel('FR')
            plt.xlabel('Time Normalized')
            plt.subplot(1,2,2)
            plt.title(str(idx[idx[:,2] == center,0:2]) +'#' + str(center)+ ' cell hub')
            plt.show()
            draw_node(df_hub,idx_type)
        
def flatten_activity(activity):  
    # Because of how reshape works we have to swap axes before reshaping. We need to have the slowest index in the end so it can be reshape in "C" Order.
    # This function take activity matrix as Time,Neurons,Events and transforms it into Time,Events,Neurones
    flat_activity = np.swapaxes(activity,1,2).reshape(-1,activity.shape[1])
    return flat_activity 

def get_hubs_corr(df,idx,hubs,activity,metadata,plot = False):
    
    #Author : BK 08/2020
    #Compute the corr of all cells in activity for a given session. Then extract mean corr value for each hubs (hubs cell vs all other cells)
    n = 1000
    
    if len(hubs) == 0:
        return 0
    if len(activity.shape) != 2:
        print('Flattenning Activity')
        activity = flatten_activity(activity)
    if plot:
        plt.figure()
        plt.title(bk.session)
        plt.xlabel('Normalized Time')
        plt.ylabel('Firing Rates')
        plt.plot(activity[:,0:5])
        
    
    activity_corr = np.corrcoef(activity.T)
    
    # Correlation of a neurons with itself is 1 but should be discarded in the analysis therefore set to NaN
    
    np.fill_diagonal(activity_corr,np.nan)
#     plt.imshow(activity_corr)
    
    BLA_Shank = bf.get_structure_shanks(bk.session,'BLA') #NEED TO KEEP ONLY BLA FOR MEAN OF THE EVERYTHING
    metadata_bla_mask = [x in BLA_Shank for x in metadata[:,2]]
    
    activity_corr_bla = activity_corr[metadata_bla_mask,:]
    activity_corr_bla = activity_corr_bla[:,metadata_bla_mask]
    
    if plot:
        plt.figure()
        sns.heatmap(abs(activity_corr_bla))
            

    #Dont really known if what I have done is correct... Took the mean of acitvity_corr. Also i'm only considering magnitude of r therefore I abs everything
    
    hubs_corr = {bk.session:np.nanmean(np.abs(activity_corr))}
    
    for hub in hubs:
            df_hub = df[df['pre_syn'] == hub]
            pre_syn_id = np.unique(df_hub['pre_syn'])
            post_syn_id = np.unique(df_hub['post_syn'])
            

            
            idx_hub = np.array([idx[(i == idx[:,2])][0,:] for i in post_syn_id])
            
            
            shank_clu_pre = np.array(idx[pre_syn_id == idx[:,2],0:2])[0]
            corr_mask_pre = np.all(shank_clu_pre == metadata[:,2:4],1)
            
            
            hub_corr = []
            for i in post_syn_id:
                shank_clu_post = np.array(idx[i == idx[:,2],0:2])
                corr_mask_post = np.all(shank_clu_post == metadata[:,2:4],1)
                hub_corr.extend(abs(activity_corr[corr_mask_pre,corr_mask_post]))
            if plot:
                plt.figure()
                sns.heatmap([hub_corr],square = True)
                plt.ylabel(str(hub))
            #Gen n fake hub in order to do bootstrap
            fake_corr = [] # Corr of the all fake hub with the same cells in the center of the hub
            for i in range(n):
                n_cells = len(df_hub['pre_syn']) #Take same number of cells than in the original hub
                cells = np.unique(metadata[metadata_bla_mask,4]) # Take only cells in the BLA
                random_cells = np.sort(np.random.choice(cells,n_cells,replace = False))
                fake_hub = {'pre_syn':df_hub['pre_syn'],'post_syn':random_cells} #Create a fake hub
                df_fake_hub = pd.DataFrame(fake_hub)
                
                post_syn_id = np.unique(df_fake_hub['post_syn'])
                
                fake_corr_hub = [] # Corr of all cells in the fake hub
                for j in post_syn_id:
                    shank_clu_post = np.array(idx[j == idx[:,2],0:2])
                    corr_mask_post = np.all(shank_clu_post == metadata[:,2:4],1)
                    
                    fake_corr_hub.extend(abs(activity_corr[corr_mask_pre,corr_mask_post]))
                
                fake_corr.append(np.nanmean(fake_corr_hub))
            if plot:
                plt.figure()
                h,_,_ = plt.hist(fake_corr,bins = 30,density = True)
                plt.xlim([0, 1])
                plt.vlines(np.nanmean(fake_corr),0,max(h),color = 'r')
                plt.vlines(np.nanmean(hub_corr),0,max(h),color = 'g')
                plt.title(str(shank_clu_pre)+str(hub) + bk.session)
                plt.xlabel('Mean correlation of FR during NREM-REM sleep')
                plt.ylabel('Probability')
                plt.legend(['Mean of Fake Hubs','Mean of Real Hub'])
                plt.show()
            
            hubs_corr.update({str(shank_clu_pre):[np.nanmean(hub_corr),np.nanmean(fake_corr)]})
#             print(np.nanmean(hub_corr))
#             print(np.nanmean(np.abs(activity_corr)))
    hubs_corr = pd.DataFrame(hubs_corr,index = ['hubs','fake'])
    return hubs_corr
    
#             print(idx_hub)
#             print(pre_syn_id,post_syn_id)
#             corr_mask = [metadata[np.all((i[0:2] == metadata[:,2:4]),1),:] for i in idx_hub]
#             print(corr_mask)

        

In [9]:
def microCircuit(path,plot = False):
    bk.set_current_session(path)
    df,idx,hubs = get_hubs()
    activity,metadata = get_activity()
    draw_hubs(df,idx,hubs,activity,metadata,plot)
    hubs_corr = get_hubs_corr(df,idx,hubs,activity,metadata,plot)
    return hubs_corr

In [10]:
a = bk.batch(microCircuit)

Batch finished in 87.67152786254883
Some session were not processed correctly
['Rat08-20130708', 'Rat08-20130710', 'Rat08-20130720', 'Rat08-20130722', 'Rat09-20140408', 'Rat10-20140704', 'Rat11-20150310', 'Rat11-20150315', 'Rat11-20150403']
14.754098360655737 %


In [50]:
hubs = []
surrogate = []
for i in a.keys():
    if np.all(a[i] == 0): continue
    for j in a[i].keys():
        if j.startswith('Rat'): 
            continue
        hubs.append(a[i][j]['hubs'])
        surrogate.append(a[i][j]['fake'])
        

df_corr = pd.DataFrame(np.array([surrogate,hubs]).T,columns=['Surrogates','Hubs'])

pal = sns.cubehelix_palette(2, rot=-.5, dark=.3)
plt.figure()
sns.swarmplot(data = df_corr,palette = pal)
plt.ylabel('Mean correlation of FR during NREM-REM sleep')
plt.show()

scipy.stats.wilcoxon(df_corr['Surrogates'],df_corr['Hubs'])

WilcoxonResult(statistic=3680.0, pvalue=0.9783326471741812)

In [41]:
for i in a.keys():
    print(i[0:3])

Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat
Rat


In [10]:
mean = []
for i in a.keys():
    if np.all((a[i] == 0)): continue #If no hubs in the session, do not compute anything on it 
    mean.extend(a[i][i].values)
print(np.mean(mean))

hubs = []
for i in a.keys():
    if np.all((a[i] == 0)): continue
    for j in a[i].keys():
        hubs.extend(a[i][j])

plt.figure()
sns.violinplot(data = [mean,hubs],inner = 'points')
plt.show()

scipy.stats.wilcoxon(mean,b)

0.10891855674275717


NameError: name 'b' is not defined

In [157]:
plt.close('all')
# for i in a:
#     if isinstance(a[i],int): continue
#     for j in a[i]:
#         if any(a[i][j]>0.3): print(i)
path = 'Z:\\Rat11\\Rat11-20150326'        
a = microCircuit(path,False)

Rat : 11 on day : 15
Working with session Rat11-20150326 @ Z:\Rat11\Rat11-20150326
Found 6 hubs
Flattenning Activity


In [101]:
a

,Rat11-20150326,[ 4 16],[ 4 17],[7 6],[8 5],[ 8 17],[12 17]
hubs,0.105997,0.091465,0.148357,0.105228,0.271911,0.490309,0.066331
fake,0.105997,0.166762,0.201002,0.192273,0.220827,0.190283,0.157979


In [49]:
path = 'Z:\Rat11\Rat11-20150326'
microCircuit(path)

Rat : 11 on day : 15
Working with session Rat11-20150326 @ Z:\Rat11\Rat11-20150326
Found 6 hubs
Flattenning Activity
   pre_syn  post_syn
0       46        62
1       46        71
2       46        82
3       46        92
[0.15404179857762804, 0.10063702301679876, 0.057574626149736005, 0.05360539374887034]


TypeError: 'numpy.float64' object is not iterable

In [11]:
# session_index = pd.read_csv('Z:/All-Rats/Billel/session_indexing.csv',sep = ';')


# for path in session_index['Path']:
#     try:
#         draw_hub(path)
#     except:
#         print('oups')

## CERTAIN NEURONES NE SONT PAS DANS FINAL TYPE

Il semblerait que dans la session # path = 'Z:\\Rat08\\Rat08-20130710' les neurones 10-8 et 10-9 n'ai pas de type ... 
En fait ces neurones ne semblent même pas exister lorsque l'on load les spikes ? Pourquoi apparaissent-il dans idx de MonoSynConvClick ? Le bout de code suivant met le problème en évidence. 

In [60]:
session_index = pd.read_csv('Z:/All-Rats/Billel/session_indexing.csv',sep = ';')
path = 'Z:\\Rat08\\Rat08-20130710'
session = path.split('\\')[2]
os.chdir(path)
print(os.getcwd())

MonoSynConvClick = scipy.io.loadmat(session +'-MonoSynConvClick.mat')

finalType = scipy.io.loadmat('Z:/All-Rats/AllRats-FinalType.mat')['finalType']
rat = session_index['Rat'][session_index['Path'] == path].values
day = session_index['Day'][session_index['Path'] == path].values

print(finalType.shape)

finalType = finalType[(finalType[:,0] == rat) & (finalType[:,1] == day),2:5]
print(finalType.shape)
idx = MonoSynConvClick['Idx']
print(idx.shape)

tout_type = np.array([finalType[np.all((i[0:2] == finalType[:,0:2]),1),2] for i in idx])
print(tout_type.shape)
print(idx.shape)

print(finalType,idx)

Z:\Rat08\Rat08-20130710
(7390, 5)
(190, 3)
(190, 3)
(190,)
(190, 3)
[[ 1  2  1]
 [ 2  2  1]
 [ 2  3  1]
 [ 2  4  1]
 [ 2  5  1]
 [ 2  6  1]
 [ 2  7  1]
 [ 2  8  1]
 [ 2  9  1]
 [ 2 10  1]
 [ 2 11  1]
 [ 2 12  1]
 [ 2 13  1]
 [ 2 14  1]
 [ 2 15  1]
 [ 2 16  1]
 [ 2 17  1]
 [ 2 18  1]
 [ 2 19  1]
 [ 2 20  2]
 [ 2 21  1]
 [ 2 22  1]
 [ 2 23  1]
 [ 3  2  1]
 [ 3  3  1]
 [ 3  4  1]
 [ 3  5  1]
 [ 3  6  1]
 [ 3  7  1]
 [ 3  8  1]
 [ 3  9  2]
 [ 5  2  1]
 [ 5  3  1]
 [ 5  4  2]
 [ 5  5  1]
 [ 5  6  1]
 [ 5  7  1]
 [ 5  8  1]
 [ 5  9  1]
 [ 5 10  1]
 [ 5 11  1]
 [ 5 12  2]
 [ 5 13  1]
 [ 5 14  1]
 [ 6  2  1]
 [ 6  3  1]
 [ 6  4  1]
 [ 6  5  2]
 [ 6  6  1]
 [ 6  7  1]
 [ 6  8  1]
 [ 7  2  1]
 [ 7  3  1]
 [ 7  4  1]
 [ 7  5  1]
 [ 7  6  1]
 [ 7  7  1]
 [ 7  8  1]
 [ 7  9  1]
 [ 7 10  1]
 [ 7 11  1]
 [ 7 12  1]
 [ 7 13  1]
 [ 7 14  1]
 [ 7 15  1]
 [ 7 16  1]
 [ 7 17  1]
 [ 7 18  1]
 [ 7 19  1]
 [ 7 20  2]
 [ 7 21  1]
 [ 7 22  1]
 [ 7 23  1]
 [ 7 24  2]
 [ 8  2  1]
 [ 8  3  1]
 [ 8  4  1]
 [ 8  5 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [7]:
path = "Z:\Rat10\Rat10-20140628"

session = path.split('\\')[2]
os.chdir(path)
activity = scipy.io.loadmat('Billel/Transitions/NREM_REM_ALL.mat')['binned']['activity'][0][0]
metadata = scipy.io.loadmat('Billel/Transitions/NREM_REM_ALL.mat')['binned']['metadata'][0][0]

finalType = scipy.io.loadmat('Z:/All-Rats/AllRats-FinalType.mat')['finalType']

flat_activity = flatten_activity(activity)
corr = np.corrcoef(flat_activity.T)

plt.figure()
plt.imshow(corr)
plt.show()

# print(metadata)

# Example of hub with most correlation on all sessions : 

In [51]:
path = 'Z:\Rat11\Rat11-20150326'
bk.set_current_session(path)
df,idx,hubs = get_hubs()
hubs = [90] #Only looking for the hub with cell 90 because it is the one swwith the greater correlation
activity,metadata = get_activity()
draw_hubs(df,idx,hubs,activity,metadata,True)
hubs_corr = get_hubs_corr(df,idx,hubs,activity,metadata)

print(hubs_corr)

Rat : 11 on day : 15
Working with session Rat11-20150326 @ Z:\Rat11\Rat11-20150326
Found 6 hubs


TypeError: get_hubs_corr() missing 1 required positional argument: 'plot'

In [19]:
plt.close('all')

In [11]:
path = 'Z:\Rat11\Rat11-20150326'
microCircuit(path)

Rat : 11 on day : 15
Working with session Rat11-20150326 @ Z:\Rat11\Rat11-20150326
Found 6 hubs
Flattenning Activity


,Rat11-20150326,[ 4 16],[ 4 17],[7 6],[8 5],[ 8 17],[12 17]
hubs,0.105997,0.091465,0.148357,0.105228,0.271911,0.490309,0.066331
fake,0.105997,0.164925,0.200356,0.193226,0.220758,0.191530,0.159738


In [31]:
a

{'Rat08-20130709':       Rat08-20130709     [6 8]     [6 9]   [ 6 10]   [ 7 22]   [ 7 28]  \
 hubs        0.093469  0.091767  0.144137  0.109573  0.223393  0.253207   
 fake        0.093469  0.072526  0.148260  0.133824  0.184764  0.184962   
 
          [8 9]   [ 8 10]   [17 17]   [18 10]  
 hubs  0.343062  0.219925  0.104018  0.125897  
 fake  0.207224  0.181880  0.159926  0.103049  ,
 'Rat08-20130711':       Rat08-20130711   [ 6 11]   [ 6 12]   [ 6 16]   [ 7 28]   [16  7]  \
 hubs        0.088843  0.166988  0.121530  0.084391  0.120989  0.151930   
 fake        0.088843  0.170073  0.123345  0.119895  0.128219  0.181066   
 
        [17  6]   [17  8]   [18  7]  
 hubs  0.082056  0.052112  0.155367  
 fake  0.120055  0.125421  0.166371  ,
 'Rat08-20130712':       Rat08-20130712     [5 8]   [ 6 10]   [15  5]   [18  2]
 hubs        0.088205  0.113098  0.114495  0.101165  0.150919
 fake        0.088205  0.143935  0.129476  0.105273  0.151450,
 'Rat08-20130713':       Rat08-20130713   [ 6